In [1]:
import os , sys
from dask import compute, persist
from dask.distributed import Client, progress
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
from sklearn.neural_network import MLPClassifier
from dask_ml.preprocessing import StandardScaler,LabelEncoder
from dask_ml.model_selection import HyperbandSearchCV ,RandomizedSearchCV,KFold
import numpy as np
import pickle
import dask.array as da
from sklearn.metrics import classification_report,matthews_corrcoef
from sklearn.linear_model import SGDClassifier
from scipy.stats import uniform, loguniform 
from dask_ml.xgboost import XGBClassifier
import pickle


# import xgboost.XGBRFClassifier as xRFC

In [2]:
import dask
print (f'Dask version {dask.__version__}')

Dask version 2.20.0


In [3]:
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='16GB')

In [4]:
client

Client Scheduler: inproc://10.228.0.65/20962/1 Dashboard: http://10.228.0.65:8787/status,Cluster Workers: 1 Cores: 4 Memory: 16.00 GB


In [5]:
PDB_BM5 = [
'1EXB','1JTD','1M27','1RKE','2A1A','2GAF','2GTP','2VXT','2W9E',
'2X9A','2YVJ','3A4S','3AAA','BAAD','3AAD','3BIW','3BX7',
'3DAW','3EO1','3EOA','3F1P','3FN1','3G6D','3H11',
'3H2V','3HI6','3HMX','3K75','3L5W','3L89','3LVK','3MXW',
'BP57','CP57','3P57','3PC8','3R9A','3RVW','3S9D','3SZK',
'3V6Z','3VLB','4DN4','4FQI','4FZA','4G6J','4G6M','4GAM',
'4GXU','4H03','4HX3','4IZ7','4JCV','4LW4','4M76'
]

In [6]:
def load_data_sets():
    all_balanced_data = dd.read_csv("../data/Clean_dataframe_balanced_all_data_ccharppi_4_march_2020_complete.csv")
    all_balanced_data = all_balanced_data.drop("class_q",axis=1)
    all_balanced_data = all_balanced_data.set_index('Conf')
#     all_balanced_data.loc["Z_1JTG_1136_M.pdb","DDG_V"]  = all_balanced_data["DDG_V"].mean()
    
    all_unbalanced_data = dd.read_csv("../data/Clean_dataframe_unbalanced_all_data_ccharppi_4_march_2020_complete.csv",dtype={'class_q': 'object'})
    all_unbalanced_data = all_unbalanced_data.drop("class_q",axis=1)
    all_unbalanced_data = all_unbalanced_data.set_index('Conf')
#     all_unbalanced_data.loc["Z_1JTG_1136_M.pdb","DDG_V"]  = all_balanced_data["DDG_V"].mean()


    Scorers_balanced_data = dd.read_csv("../data/Clean_dataframe_balanced_scorers_set_feb_12_2021.csv")
    Scorers_balanced_data = Scorers_balanced_data.set_index('Conf')
    Scorers_balanced_data = Scorers_balanced_data.dropna()

    Scorers_unbalanced_data = dd.read_csv("../data/Clean_dataframe_unbalanced_scorers_set_feb_12_2021.csv")
    Scorers_unbalanced_data = Scorers_unbalanced_data.set_index('Conf')
    Scorers_unbalanced_data = Scorers_unbalanced_data.dropna()

    X_train = all_balanced_data[~all_balanced_data["idx"].isin(PDB_BM5) ]
    y_train = all_balanced_data[~all_balanced_data["idx"].isin(PDB_BM5) ]["label_binary"].astype('bool')

            ## data set for less than 5 
    X_val = all_balanced_data[all_balanced_data["idx"].isin(PDB_BM5) ]
    y_val = all_balanced_data[all_balanced_data["idx"].isin(PDB_BM5) ]["label_binary"].astype('bool')
    #         print (X_test.size,y_test.size)
            ## data set for less than 5 
    X_val_u = all_unbalanced_data[all_unbalanced_data["idx"].isin(PDB_BM5) ]
    y_val_u = all_unbalanced_data[all_unbalanced_data["idx"].isin(PDB_BM5) ]["label_binary"].astype('bool')
    
    X_test = Scorers_balanced_data
    y_test = Scorers_balanced_data["binary_label"].astype('bool')
    
    X_test_u = Scorers_unbalanced_data
    y_test_u = Scorers_unbalanced_data["binary_label"].astype('bool')
    
    X_test_u = X_test_u.rename(columns={'NIS Polar' :'Nis_Polar',
                                  'Nis Apolar':'Nis_Apolar',
                                  'BSA Apolar':'BSA_Apolar',
                                  'BSA Polar' :'BSA_Polar',
                                'binary_label':'label_binary'
                            })
    X_test= X_test.rename(columns={'NIS Polar' :'Nis_Polar',
                                  'Nis Apolar':'Nis_Apolar',
                                  'BSA Apolar':'BSA_Apolar',
                                  'BSA Polar' :'BSA_Polar',
                                   'binary_label':'label_binary'
                          })
    
#     for x in X_val_u.columns:
#         if x not in X_test.columns:
#             print (x)
    return X_train, y_train , X_val, y_val, X_test, y_test ,X_val_u, y_val_u, X_test_u, y_test_u 

In [7]:
def scaling_data_selected(X_train,X_test,X_test_unbalanced):
#     scaler = MinMaxScaler()
    scaler = StandardScaler()
    selected_feat = ['AP_DFIRE2','AP_PISA','AP_T1','AP_T2','CP_MJ3h','SIPPER','ELE','VDW','PYDOCK_TOT','AP_dDFIRE']


#     selected_feat = ['CONSRANK_val','AP_GOAP_DF','CP_TD','CP_D1','CP_HLPL',
#                       'DDG_V','CP_MJ3h','PYDOCK_TOT','ELE','CP_SKOIP',
#                       'SIPPER','AP_DFIRE2','AP_dDFIRE','AP_PISA','CP_RMFCA',
# #                      'CP_TB','AP_DARS','CP_BT'
#                      ]
    
#     selected_feat = ['CONSRANK_val','CP_HLPL','CP_MJ3h','DDG_V','CP_RMFCA','AP_GOAP_DF','CP_Qp','CP_TD','CP_SKOIP','CP_TB','CP_TSC','PYDOCK_TOT','SIPPER',
# 'CP_BT','CP_MJ2h','AP_DFIRE2','CP_RMFCEN1','AP_DARS','AP_PISA','BSA_Apolar','CP_BFKV','AP_dDFIRE','CP_RMFCEN2','CP_ZS3DC_MIN',
# 'AP_DDG_U','AP_DDG_W','cips_AlAr','CP_MJPL','CP_SKOb','CP_TEl','CP_TS','PROPNSTS','AP_MPS','CP_D1','FIREDOCK','AlAr','ArAr']

    X_train = X_train[selected_feat]
    X_test = X_test[selected_feat]
    X_test_unbalanced= X_test_unbalanced[selected_feat]


    scaler.fit(X_train)
    scaled_train = scaler.transform(X_train)
    scaled_test = scaler.transform(X_test)
    scaled_test_u = scaler.transform(X_test_unbalanced)
    pickle.dump(scaler, open("ScalerBM4.pickle.dat", "wb"))

    return scaled_train,scaled_test,scaled_test_u

In [8]:
def scaling_data(X_train,X_test,X_test_unbalanced):
#     scaler = MinMaxScaler()
    features = ['idx','class_q','pdb1','chains_pdb1','pdb2','chains_pdb2',
                'label_binary','DQ_val','binary_label','identification','labels']
    scaler = StandardScaler()

    for x in features :
        if x in X_train.columns:
            X_train= X_train.drop(x,axis=1)            
    for x in features :
        if x in X_test.columns: 
                X_test = X_test.drop(x,axis=1)
            
    for x in features :
        if x in X_test_unbalanced.columns:
            X_test_unbalanced= X_test_unbalanced.drop(x,axis=1)

    scaler.fit(X_train)
    scaled_train = scaler.transform(X_train)
    scaled_test = scaler.transform(X_test)
    scaled_test_u = scaler.transform(X_test_unbalanced)
    return scaled_train,scaled_test,scaled_test_u

In [9]:
X_train, y_train , X_val, y_val, X_test, y_test ,X_val_u, y_val_u, X_test_u, y_test_u  = load_data_sets()

In [10]:
params = {
    "hidden_layer_sizes": [

        (24, ),
        (12, 12),
        (6, 6, 6, 6),
        (4, 4, 4, 4, 4, 4),
        (12, 6, 3, 3),
    ],
    "activation": ["relu", "logistic", "tanh"],
    "alpha": np.logspace(-6, -3, num=1000),  # cnts
    "batch_size": [16, 32, 64, 128, 256, 512],
}


In [11]:
estimator = MLPClassifier()

In [12]:
# X_train_w, X_test_w, X_test_u_w = scaling_data(X_train, X_test, X_test_u)
# X_train_w, X_val_w, X_val_u_w = scaling_data(X_train, X_val, X_val_u)

X_train_fs, X_test_fs, X_test_u_fs = scaling_data_selected(X_train, X_test, X_test_u)
X_train_fs, X_val_fs, X_val_u_fs = scaling_data_selected(X_train, X_val, X_val_u)

In [13]:
n_examples = 4 * len(X_train_fs)  # 20 passes through dataset for best model
n_params = 8  # sample about 300 parameters

# inputs to hyperband
max_iter = n_params
chunk_size = n_examples // n_params

In [14]:
X_train_fs.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 10 entries, AP_DFIRE2 to AP_dDFIRE
dtypes: float64(10)

In [15]:
X_train_fs.describe()

,AP_DFIRE2,AP_PISA,AP_T1,AP_T2,CP_MJ3h,SIPPER,ELE,VDW,PYDOCK_TOT,AP_dDFIRE
npartitions=1,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...


In [16]:
X_train_fs = X_train_fs.to_dask_array()


In [17]:
X_train_fs.compute_chunk_sizes()

dask.array<values, shape=(55748, 10), dtype=float64, chunksize=(31885, 10), chunktype=numpy.ndarray>

In [18]:
X_train_fs = X_train_fs.rechunk(chunks=chunk_size)

In [19]:
# X_train_fs

In [20]:
y_train = y_train.to_dask_array()

In [21]:
y_train.compute_chunk_sizes()

dask.array<values, shape=(55748,), dtype=bool, chunksize=(31885,), chunktype=numpy.ndarray>

In [22]:
y_train = y_train.rechunk(chunks=chunk_size)

In [23]:
# y_train

In [24]:
# Create the random search model
rsnn = HyperbandSearchCV(estimator=estimator, parameters=params ,verbose=True,
                         max_iter=max_iter,patience=True,
                         random_state=32,aggressiveness=4)

In [25]:
rsnn.metadata["partial_fit_calls"]

30

In [26]:
np.random.seed(seed = 101) 

rsnn.fit(X_train_fs,y_train,classes=[True,False])

[CV, bracket=1] creating 4 models
[CV, bracket=0] creating 2 models
[CV, bracket=1] For training there are between 22299 and 22299 examples in each chunk
[CV, bracket=0] For training there are between 22299 and 22299 examples in each chunk
[CV, bracket=0] validation score of 0.7803 received after 1 partial_fit calls
[CV, bracket=1] validation score of 0.7711 received after 1 partial_fit calls
[CV, bracket=1] validation score of 0.7766 received after 2 partial_fit calls
[CV, bracket=0] validation score of 0.7787 received after 3 partial_fit calls
[CV, bracket=1] validation score of 0.7804 received after 4 partial_fit calls
[CV, bracket=0] validation score of 0.7757 received after 5 partial_fit calls
[CV, bracket=1] validation score of 0.7809 received after 6 partial_fit calls


HyperbandSearchCV(aggressiveness=4, estimator=MLPClassifier(), max_iter=8,
                  parameters={'activation': ['relu', 'logistic', 'tanh'],
                              'alpha': array([1.00000000e-06, 1.00693863e-06, 1.01392541e-06, 1.02096066e-06,
       1.02804473e-06, 1.03517796e-06, 1.04236067e-06, 1.04959323e-06,
       1.05687597e-06, 1.06420924e-06, 1.07159340e-06, 1.07902879e-06,
       1.08651577e-06, 1.09405471e-06, 1...
       9.01477631e-04, 9.07732653e-04, 9.14031075e-04, 9.20373200e-04,
       9.26759330e-04, 9.33189772e-04, 9.39664831e-04, 9.46184819e-04,
       9.52750047e-04, 9.59360829e-04, 9.66017480e-04, 9.72720319e-04,
       9.79469667e-04, 9.86265846e-04, 9.93109181e-04, 1.00000000e-03]),
                              'batch_size': [16, 32, 64, 128, 256, 512],
                              'hidden_layer_sizes': [(24,), (12, 12),
                                                     (6, 6, 6, 6),
                                                     (4, 4,

In [27]:
print (rsnn.best_params_)

{'hidden_layer_sizes': (24,), 'batch_size': 32, 'alpha': 1.1805165285688055e-06, 'activation': 'relu'}


In [28]:
classifier = rsnn.best_estimator_

In [29]:
y_pred= classifier.predict(X_val_u_fs)

In [30]:
print (classification_report( y_true=y_val_u ,y_pred= y_pred))

              precision    recall  f1-score   support

       False       0.96      0.76      0.85    146541
        True       0.16      0.59      0.25     11226

    accuracy                           0.75    157767
   macro avg       0.56      0.67      0.55    157767
weighted avg       0.90      0.75      0.80    157767



In [31]:
y_pred= classifier.predict(X_test_u_fs)

In [32]:
print (classification_report( y_true=y_test_u ,y_pred= y_pred))

              precision    recall  f1-score   support

       False       0.99      0.43      0.60     14283
        True       0.19      0.97      0.31      1899

    accuracy                           0.50     16182
   macro avg       0.59      0.70      0.46     16182
weighted avg       0.90      0.50      0.57     16182



In [33]:
estimator = SGDClassifier() 

In [34]:
#### SGD params 
params = { 
    "l1_ratio": uniform(0, 1), 
    "alpha": loguniform(1e-5, 1e-1), 
    "penalty": ["l2", "l1", "elasticnet"], 
    "learning_rate": ["invscaling", "adaptive","optimal"], 
    "power_t": uniform(0, 1), 
    "average": [True, False], 
    "loss":["squared_loss", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"], 
    "epsilon": loguniform(1e-5, 1e-1), 
    "eta0":np.logspace(-6, -3, num=1000)
} 


In [35]:
rsnn = HyperbandSearchCV(estimator=estimator, parameters=params ,verbose=True,
                         max_iter=max_iter,patience=True,
                         random_state=32,aggressiveness=4)

In [36]:
rsnn.metadata["partial_fit_calls"]

30

In [37]:
np.random.seed(seed = 101) 

rsnn.fit(X_train_fs,y_train,classes=[True,False])

[CV, bracket=1] creating 4 models
[CV, bracket=0] creating 2 models
[CV, bracket=1] For training there are between 22299 and 22299 examples in each chunk
[CV, bracket=0] For training there are between 22299 and 22299 examples in each chunk
[CV, bracket=0] validation score of 0.7703 received after 1 partial_fit calls
[CV, bracket=1] validation score of 0.7608 received after 1 partial_fit calls
[CV, bracket=1] validation score of 0.7735 received after 2 partial_fit calls
[CV, bracket=0] validation score of 0.7779 received after 3 partial_fit calls
[CV, bracket=1] validation score of 0.7749 received after 4 partial_fit calls
[CV, bracket=0] validation score of 0.7800 received after 5 partial_fit calls
[CV, bracket=1] validation score of 0.7755 received after 6 partial_fit calls


HyperbandSearchCV(aggressiveness=4, estimator=SGDClassifier(), max_iter=8,
                  parameters={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x15670c790>,
                              'average': [True, False],
                              'epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x15670ce80>,
                              'eta0': array([1.00000000e-06, 1.00693863e-06, 1.01392541e-06, 1.02096066e-06,
       1.02804473e-06, 1.0351...
                              'l1_ratio': <scipy.stats._distn_infrastructure.rv_frozen object at 0x15670c520>,
                              'learning_rate': ['invscaling', 'adaptive',
                                                'optimal'],
                              'loss': ['squared_loss', 'huber',
                                       'epsilon_insensitive',
                                       'squared_epsilon_insensitive'],
                              'penalty': ['l2', 'l1', 'elasticnet']

In [38]:
print ( rsnn.best_params_ )

{'alpha': 0.027261959556018345, 'average': False, 'epsilon': 0.0004832278256276128, 'eta0': 0.0007531420165974368, 'l1_ratio': 0.9556565489917677, 'learning_rate': 'invscaling', 'loss': 'squared_epsilon_insensitive', 'penalty': 'elasticnet', 'power_t': 0.35360111371492264}


In [39]:
classifier = rsnn.best_estimator_

In [40]:
y_pred= classifier.predict(X_val_u_fs)

In [41]:
print (classification_report( y_true=y_val_u ,y_pred= y_pred))

              precision    recall  f1-score   support

       False       0.96      0.75      0.84    146541
        True       0.16      0.61      0.25     11226

    accuracy                           0.74    157767
   macro avg       0.56      0.68      0.55    157767
weighted avg       0.90      0.74      0.80    157767



In [42]:
y_pred= classifier.predict(X_test_u_fs)

In [43]:
print (classification_report( y_true=y_test_u ,y_pred= y_pred))

              precision    recall  f1-score   support

       False       0.99      0.40      0.57     14283
        True       0.18      0.98      0.30      1899

    accuracy                           0.47     16182
   macro avg       0.59      0.69      0.44     16182
weighted avg       0.90      0.47      0.54     16182



In [44]:
xgb = XGBClassifier()

In [45]:
# params = { 
# "learning_rate" :[0.0001, 0.001, 0.01, 0.1], 
#  "n_estimators":np.linspace(start = 100, stop = 1000, num = 5),
# #  "max_depth":np.linspace(1, 11, num = 10),
# #  "min_child_weight": np.linspace(1, 11, num = 10),
# #  "gamma":[0,2,4,6,8],
# #  "subsample":np.linspace(0,1,num=10),
# #  "colsample_bytree":np.linspace(0,1,num=10),
#  "objective": ['binary:logistic','binary:logitraw','binary:hinge']
# }


In [46]:
# label_encoded_y = LabelEncoder().fit_transform(y_train)


In [47]:
# rsnn.metadata["partial_fit_calls"]

In [48]:
np.random.seed(seed = 101) 
xgb.fit(X_train_fs,y_train)
# rsnn.fit(X_train_fs,y_train,classes=[True,False])
# random_S.fit(X_train_fs.compute(),y_train.compute())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='approx', validate_parameters=1, verbosity=None)

In [49]:
y_pred= xgb.predict(X_test_u_fs.to_dask_array())

In [50]:
print (classification_report( y_true=y_test_u ,y_pred= y_pred.compute()))

              precision    recall  f1-score   support

       False       0.97      0.56      0.71     14283
        True       0.21      0.89      0.34      1899

    accuracy                           0.60     16182
   macro avg       0.59      0.72      0.52     16182
weighted avg       0.88      0.60      0.67     16182



In [51]:
y_pred= xgb.predict(X_val_u_fs.to_dask_array())

In [52]:
print (classification_report( y_true=y_val_u ,y_pred= y_pred.compute()))

              precision    recall  f1-score   support

       False       0.96      0.79      0.86    146541
        True       0.17      0.59      0.27     11226

    accuracy                           0.77    157767
   macro avg       0.57      0.69      0.57    157767
weighted avg       0.91      0.77      0.82    157767



In [53]:
# xgb = XGBClassifier()

In [54]:
# np.random.seed(seed = 101) 

# # rsnn = HyperbandSearchCV(estimator=xgb, parameters=params ,verbose=True,
# #                          max_iter=max_iter,patience=True,
# #                          random_state=32,aggressiveness=4)
# # kfold = KFold(n_splits=10, shuffle=True, random_state=101)

# random_S = RandomizedSearchCV(estimator = xgb,
#                               param_distributions = params,
#                               scoring="neg_log_loss",
# #                               cv = kfold,
#                               random_state=101, 
#                               n_jobs = -1)

In [55]:
# type(label_encoded_y)

In [56]:
# random_S.fit(X_train_fs,y_train)


In [57]:
# print( random_S.best_estimator_ ) 

In [58]:
np.random.seed(seed = 101) 
# xgb.fit(X_train_fs,y_train)
# rsnn.fit(X_train_fs,y_train,classes=[True,False])
# random_S.fit(X_train_fs,y_train)
for x in  ['binary:logistic','binary:logitraw','binary:hinge']:
    xgb = XGBClassifier(objective=x)
    xgb.fit(X_train_fs,y_train)
    y_pred= xgb.predict(X_test_u_fs.to_dask_array())
    print (classification_report( y_true=y_test_u ,y_pred= y_pred.compute()))
    print ()
    


              precision    recall  f1-score   support

       False       0.97      0.56      0.71     14283
        True       0.21      0.89      0.34      1899

    accuracy                           0.60     16182
   macro avg       0.59      0.72      0.52     16182
weighted avg       0.88      0.60      0.67     16182


              precision    recall  f1-score   support

       False       0.96      0.63      0.76     14283
        True       0.23      0.82      0.35      1899

    accuracy                           0.65     16182
   macro avg       0.59      0.72      0.56     16182
weighted avg       0.88      0.65      0.71     16182


              precision    recall  f1-score   support

       False       0.95      0.57      0.71     14283
        True       0.19      0.78      0.31      1899

    accuracy                           0.59     16182
   macro avg       0.57      0.67      0.51     16182
weighted avg       0.86      0.59      0.66     16182




In [59]:
np.random.seed(seed = 101) 
# xgb.fit(X_train_fs,y_train)
# rsnn.fit(X_train_fs,y_train,classes=[True,False])
# random_S.fit(X_train_fs,y_train)
for x in np.linspace(start = 100, stop = 1000, num = 5) :
    print(x)
    xgb = XGBClassifier(objective='binary:logitraw', n_estimators= int(x) )
    
    xgb.fit(X_train_fs,y_train)
    y_pred= xgb.predict(X_test_u_fs.to_dask_array())
    print (classification_report( y_true=y_test_u ,y_pred= y_pred.compute()))
    print ()

100.0
              precision    recall  f1-score   support

       False       0.96      0.63      0.76     14283
        True       0.23      0.82      0.35      1899

    accuracy                           0.65     16182
   macro avg       0.59      0.72      0.56     16182
weighted avg       0.88      0.65      0.71     16182


325.0
              precision    recall  f1-score   support

       False       0.97      0.63      0.76     14283
        True       0.23      0.85      0.36      1899

    accuracy                           0.65     16182
   macro avg       0.60      0.74      0.56     16182
weighted avg       0.88      0.65      0.71     16182


550.0
              precision    recall  f1-score   support

       False       0.97      0.62      0.76     14283
        True       0.23      0.86      0.36      1899

    accuracy                           0.65     16182
   macro avg       0.60      0.74      0.56     16182
weighted avg       0.88      0.65      0.71     16182


In [60]:
np.random.seed(seed = 101) 
# xgb.fit(X_train_fs,y_train)
# rsnn.fit(X_train_fs,y_train,classes=[True,False])
# random_S.fit(X_train_fs,y_train)
learning_rate = [0.0003, 0.003, 0.03, 0.3]
for x in  learning_rate:
    xgb = XGBClassifier(objective='binary:logitraw',learning_rate=x)
    xgb.fit(X_train_fs,y_train)
    y_pred= xgb.predict(X_test_u_fs.to_dask_array())
    print (classification_report( y_true=y_test_u ,y_pred= y_pred.compute()))
    print ()

/Users/barradd/anaconda2/envs/dask-examples/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       False       0.88      1.00      0.94     14283
        True       0.00      0.00      0.00      1899

    accuracy                           0.88     16182
   macro avg       0.44      0.50      0.47     16182
weighted avg       0.78      0.88      0.83     16182




/Users/barradd/anaconda2/envs/dask-examples/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       False       0.88      1.00      0.94     14283
        True       0.00      0.00      0.00      1899

    accuracy                           0.88     16182
   macro avg       0.44      0.50      0.47     16182
weighted avg       0.78      0.88      0.83     16182


              precision    recall  f1-score   support

       False       0.98      0.54      0.69     14283
        True       0.21      0.92      0.34      1899

    accuracy                           0.58     16182
   macro avg       0.59      0.73      0.52     16182
weighted avg       0.89      0.58      0.65     16182


              precision    recall  f1-score   support

       False       0.96      0.63      0.76     14283
        True       0.23      0.82      0.35      1899

    accuracy                           0.65     16182
   macro avg       0.59      0.72      0.56     16182
weighted avg       0.88      0.65      0.71     16182




In [61]:
xgb = XGBClassifier(objective='binary:logitraw')

In [62]:
xgb.fit(X_train_fs,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='binary:logitraw', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='approx', validate_parameters=1, verbosity=None)

In [69]:
y_pred= xgb.predict(X_val_u_fs.to_dask_array())
cr = classification_report( y_true=y_val_u ,y_pred= y_pred.compute())
print (cr )

              precision    recall  f1-score   support

       False       0.95      0.85      0.90    146541
        True       0.20      0.47      0.28     11226

    accuracy                           0.83    157767
   macro avg       0.58      0.66      0.59    157767
weighted avg       0.90      0.83      0.86    157767



In [70]:
y_pred= xgb.predict(X_test_u_fs.to_dask_array())
cr = classification_report( y_true=y_test_u ,y_pred= y_pred.compute())
print (cr)

              precision    recall  f1-score   support

       False       0.96      0.63      0.76     14283
        True       0.23      0.82      0.35      1899

    accuracy                           0.65     16182
   macro avg       0.59      0.72      0.56     16182
weighted avg       0.88      0.65      0.71     16182



In [65]:
pickle.dump(xgb, open("XgbClassBM4.pickle.dat", "wb"))

In [ ]:
xgb.predict_proba